## Aula 05

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

seed = 4321
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#dados = pd.read_csv("https://github.com/alura-cursos/imersao-dados-2-2020/blob/master/MICRODADOS_ENEM_2019_SAMPLE_43278.csv?raw=true")

# Armazenando os dados do arquivo .csv em um Dataframe chamado dados
dados = pd.read_csv("dataset/MICRODADOS_ENEM_2019.csv")
dados.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_TEMPO_ADICIONAL,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,IN_GUIA_INTERPRETE,IN_COMPUTADOR,IN_CADEIRA_ESPECIAL,IN_CADEIRA_CANHOTO,IN_CADEIRA_ACOLCHOADA,IN_PROVA_DEITADO,IN_MOBILIARIO_OBESO,IN_LAMINA_OVERLAY,IN_PROTETOR_AURICULAR,IN_MEDIDOR_GLICOSE,IN_MAQUINA_BRAILE,IN_SOROBAN,IN_MARCA_PASSO,IN_SONDA,IN_MEDICAMENTOS,IN_SALA_INDIVIDUAL,IN_SALA_ESPECIAL,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001004661,2019,1506138,Redenção,15,PA,17,M,1,3,1,1506138.0,Redenção,15.0,PA,3,0,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1506138,Redenção,15,PA,1,1,1,1,506.0,509.0,514.0,517.0,435.6,512.3,488.6,432.4,DACDDEDEDAADCBCBDBDBBBDCCACDECADCCCBCBACCCCEA,ACAEAAACABEBABAADCEEEDE*CBDCCCADADCCCBEBBBBDB,99999CCCABBCAADDBCEBCCADBEEBDECBAABDEACACAEABB...,DEEDCAECDDEEECBCBECABEBAECBBCDAECAEBBBBBDCCDB,1,DEADBAAAEBEECEBCBCBCBDADAEABCEDDDDADCBEECACBC,ACACEEBCCBABADBBBACDBBACCCCADCEBADCBEEDBBEADB,EBBADCABDABACBCEBDEEAAADDBECDECDDBADBCDAAECBCC...,AADDDBEEEBEDDBEBACABCDBABECECACAECDCBDCCEDCDA,1.0,60.0,100.0,80.0,80.0,100.0,420.0,B,B,A,A,4,B,A,C,B,A,C,B,A,A,A,A,A,A,A,B,A,C,A,B,B
1,190001004674,2019,1504208,Marabá,15,PA,23,M,1,3,1,1504208.0,Marabá,15.0,PA,1,3,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1504208,Marabá,15,PA,0,1,1,0,NaN,508.0,512.0,NaN,NaN,409.3,375.3,NaN,NaN,ABBCEADCEBAAACACEDCDCDACCCACDCBCABCAACBDBDEDE,99999ABCEBCEDDECEEEBBCCEECCABDABEEBCDEAEBBEAEA...,NaN,1,NaN,CBABADBBCEEEBCBADCBEEDBBEADBBACDBBACCCCADACAC,BBEDABDACACBABAECBBCCADCEBDBBCDDEEAAADDBECDECA...,NaN,1.0,80.0,100.0,80.0,80.0,60.0,400.0,C,C,A,A,4,B,A,B,C,A,A,B,B,B,A,A,A,A,B,A,A,B,A,A,B
2,190001004722,2019,1501402,Belém,15,PA,35,F,2,1,1,1501402.0,Belém,15.0,PA,1,12,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1501402,Belém,15,PA,1,1,1,1,503.0,508.0,512.0,515.0,423.2,499.1,441.0,427.2,DDEACAECCC*BCDBAACDACCBAEBBCEEEAECABBCECBBEAA,CDCAACCAAADEACBBEBAAEECDABDEBAEAABEDCABABAABC,99999ECBCECEBBABECCCECECCEECCDECEEEDADDDDECEDD...,DDBEADEBBCADCEEACABECBCEBADECADCEDAACBDABABCE,1,AAECACDEADCBCDDDBCBDADAEABCEBABEEBCBEECEBDADC,CBABADBBCEEEBCBADCBEEDBBEADBBACDBBACCCCADACAC,BBEDABDACACBABAECBBCCADCEBDBBCDDEEAAADDBECDECA...,EEEADBEBACABCDBABECECACDCBDCCEDCDABEDECDDDBAA,1.0,100.0,120.0,120.0,100.0,120.0,560.0,C,A,B,B,5,C,A,B,C,A,A,B,A,B,A,A,A,A,B,A,B,B,A,A,B
3,19000100473

In [2]:
# Delimitando quais colunas possuem provas
provas = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO"]

# Criando uma coluna no DataFrame contendo a soma das cinco provas
dados["NU_NOTA_TOTAL"] = dados[provas].sum(axis=1)
provas.append("NU_NOTA_TOTAL")

In [3]:
# Removendo os inscritos que possuem nota total 0
dados_sem_notas_zero = dados.query("NU_NOTA_TOTAL != 0")

In [6]:
# Prevendo a nota de MT baseada nas outras notas
provas_entrada = ["NU_NOTA_CH","NU_NOTA_LC", "NU_NOTA_CN","NU_NOTA_REDACAO"]
prova_saida = "NU_NOTA_MT"

# Criando um DataFrame apenas com as provas, sem valores NaN
dados_sem_notas_zero = dados_sem_notas_zero[provas].dropna()

# Criando um DataFrame apenas com as colunas da lista provas_entrada
notas_entrada = dados_sem_notas_zero[provas_entrada]

# Criando um DataFrame apenas com o valor contido em provas_saida
notas_saida = dados_sem_notas_zero[prova_saida]

In [7]:
# Determinando o X (entrada) e o y (saída) do modelo
x = notas_entrada
y = notas_saida 

In [8]:
# Criando a separação entre os dados de teste e os dados de treino (teste com 25% do tamanho)
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25, random_state=seed)

In [10]:
# Criando e ajustando o modelo LinearSVR
modelo = LinearSVR(random_state=seed)
modelo.fit(x_treino, y_treino)

# Realizando a predição e obtendo o valor do erro médio quadrado
predicoes_matematica = modelo.predict(x_teste)
mean_squared_error(y_teste, predicoes_matematica)

18324.11972400897

In [12]:
# Criando e ajustando o modelo DecisionTreeRegressor
modelo_arvore = DecisionTreeRegressor(max_depth=3)
modelo_arvore.fit(x_treino, y_treino)

# Realizando a predição e obtendo o valor do erro médio quadrado
predicoes_matematica_arvore = modelo_arvore.predict(x_teste)
mean_squared_error(y_teste, predicoes_matematica_arvore)

6027.166905272975

In [18]:
# Criando o modelo DecisionTreeRegressor com max_depth=2
modelo_arvore = DecisionTreeRegressor(max_depth=2)

# Formulando uma validação cruzada com cv=10 (o modelo é ajustado neste passo)
resultados = cross_validate(modelo_arvore, x, y, cv= 10, scoring="neg_mean_squared_error")

# Calculando a media dos valores obtidos durante a validação cruzada
media = (resultados["test_score"]*-1).mean()
media

6558.227664658771

In [20]:
# Criando o modelo DecisionTreeRegressor com max_depth=2
modelo_arvore = DecisionTreeRegressor(max_depth=2)

# Formulando uma validação cruzada com cv=10 (o modelo é ajustado neste passo)
resultados = cross_validate(modelo_arvore, x, y, cv= 10, scoring="neg_mean_squared_error")

# Calculando a media dos valores obtidos durante a validação cruzada
media = (resultados["test_score"]*-1).mean()

# Calculando o desvio padrão entre os valores obtidos durante a validação cruzada
desvio_padrao = (resultados["test_score"]*-1).std()

# Delimitando os limites inferiores e superiores dos valores obtidos
lim_inferior = media - (2 * desvio_padrao)
lim_superior = media + (2 * desvio_padrao)

print(f"Intervalo de confiança {lim_inferior} - {lim_superior}")

Intervalo de confiança 6015.722549536807 - 7100.732779780731


In [22]:
# Valores retornados pelo modelo utilizando a validação cruzada com cv=10
resultados["test_score"]*-1

array([6118.6911947 , 6536.09830186, 6338.64260896, 6456.70685686,
       6234.97203593, 6794.25120283, 6713.45694262, 6693.04880297,
       6610.58461646, 7085.82408339])

In [23]:
# Função que calcula justamente o erro médio quadrado dos resultados obtidos na validação cruzada
def calcula_mse(resultados):
    media = (resultados["test_score"]*-1).mean()
    desvio_padrao = (resultados["test_score"]*-1).std()
    lim_inferior = media - (2*desvio_padrao)
    lim_superior = media + (2*desvio_padrao)
    print(f"Intervalo de confiança {lim_inferior} - {lim_superior}")

In [26]:
# Setando o seed para 1232 nesta célula
seed = 1232
np.random.seed(seed)

# Usando KFold para embaralhar as partes da validação cruzada (removendo da ordem 1, 2, 3, ..., 10)
partes = KFold(n_splits=10, shuffle=True)

# Criando o modelo DecisionTreeRegressor com max_depth=3
modelo_arvore = DecisionTreeRegressor(max_depth=3)

# Formulando uma validação cruzada com cv=10 mas agora embaralhado (o modelo é ajustado neste passo)
resultados = cross_validate(modelo_arvore, x, y, cv=partes, scoring="neg_mean_squared_error")

# Calculando o intervalo de confiança dos resultados da validação cruzada
calcula_mse(resultados)

Intervalo de confiança 5858.741842392759 - 6250.793292491384


In [33]:
def regressor_arvore(nivel):
    seed = 1232
    np.random.seed(seed)
    partes = KFold(n_splits=20, shuffle=True)
    modelo_arvore = DecisionTreeRegressor(max_depth=nivel)
    resultados = cross_validate(modelo_arvore, x, y, cv=partes, scoring="neg_mean_squared_error", return_train_score=True)
    print(f"Treino = {(resultados['train_score']*-1).mean().round(2)} | Teste = {(resultados['test_score']*-1).mean().round(2)}")

regressor_arvore(4)

Treino = 5763.99 | Teste = 5827.3


In [34]:
# Analisando os valores obtidos em 20 profundidades diferentes (overfitting com max_depth >= 9)
for i in range(1,21):
    regressor_arvore(i)

Treino = 7844.92 | Teste = 7874.83
Treino = 6528.49 | Teste = 6571.73
Treino = 6022.06 | Teste = 6063.79
Treino = 5763.99 | Teste = 5827.3
Treino = 5602.45 | Teste = 5694.05
Treino = 5475.18 | Teste = 5595.64
Treino = 5374.84 | Teste = 5537.95
Treino = 5283.1 | Teste = 5527.78
Treino = 5176.56 | Teste = 5587.16
Treino = 5037.73 | Teste = 5717.07
Treino = 4855.5 | Teste = 5888.6
Treino = 4626.36 | Teste = 6129.21
Treino = 4351.13 | Teste = 6425.18
Treino = 4037.93 | Teste = 6757.44
Treino = 3700.42 | Teste = 7103.06
Treino = 3344.66 | Teste = 7495.57
Treino = 2982.04 | Teste = 7916.25
Treino = 2626.56 | Teste = 8331.2
Treino = 2283.03 | Teste = 8681.32
Treino = 1962.31 | Teste = 9013.89
